### Install Sqlite3

```shell
sudo apt-get install sqlite3 sqlite3-doc
# Optional
sudo apt install sqlitebrowser
```

### Create Database

In [1]:
# sqlite3_createdb.py
import os
import sqlite3
db_filename="todo.db"
db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print("Need to create schema")
else:
    print("Database exists,assume schema does,too.")
    
conn.close()

Database exists,assume schema does,too.


### Initial Database With Schema File

In [5]:
# sqlite3_create_schema.py
# create schema with todo_schema.sql
import os
import sqlite3

db_filename = "todo.db"
schema_filename = "todo_schema.sql"

db_is_new = not os.path.exists(db_filename)

with sqlite3.connect(db_filename) as conn:
    if db_is_new:
        print("Create schema")
        with open(schema_filename,'rt') as f:
            schema = f.read()
        conn.executescript(schema)
        
        print("Inserting initial data")
        
        conn.executescript("""
        insert into project (name, description, deadline)
        values ('pymotw', 'Python Module of the Week',
                '2016-11-01');

        insert into task (details, status, deadline, project)
        values ('write about select', 'done', '2016-04-25',
                'pymotw');

        insert into task (details, status, deadline, project)
        values ('write about random', 'waiting', '2016-08-22',
                'pymotw');

        insert into task (details, status, deadline, project)
        values ('write about sqlite3', 'active', '2017-07-31',
                'pymotw');
        """
        )
    else:
        print("Database exists,assume schema does,too.")

Create schema
Inserting initial data


In [7]:
# You can check the initial data with command
!sqlite3 todo.db "select * from task"

1|1|write about select|done|2016-04-25||pymotw
2|1|write about random|waiting|2016-08-22||pymotw
3|1|write about sqlite3|active|2017-07-31||pymotw


### Retrieving Data From Database

In [8]:
# sqlite3_select_tasks.py
import sqlite3

db_filename = 'todo.db'

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()

    cursor.execute("""
    select id, priority, details, status, deadline from task
    where project = 'pymotw'
    """)

    for row in cursor.fetchall():
        task_id, priority, details, status, deadline = row
        print('{:2d} [{:d}] {:<25} [{:<8}] ({})'.format(
            task_id, priority, details, status, deadline))

 1 [1] write about select        [done    ] (2016-04-25)
 2 [1] write about random        [waiting ] (2016-08-22)
 3 [1] write about sqlite3       [active  ] (2017-07-31)


In [9]:
# sqlite3_select_variations.py
# fetch one or specify number of items from database
import sqlite3

db_filename = 'todo.db'

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()

    cursor.execute("""
    select name, description, deadline from project
    where name = 'pymotw'
    """)
    name, description, deadline = cursor.fetchone()

    print('Project details for {} ({})\n  due {}'.format(
        description, name, deadline))

    cursor.execute("""
    select id, priority, details, status, deadline from task
    where project = 'pymotw' order by deadline
    """)

    print('\nNext 5 tasks:')
    for row in cursor.fetchmany(5):
        task_id, priority, details, status, deadline = row
        print('{:2d} [{:d}] {:<25} [{:<8}] ({})'.format(
            task_id, priority, details, status, deadline))

Project details for Python Module of the Week (pymotw)
  due 2016-11-01

Next 5 tasks:
 1 [1] write about select        [done    ] (2016-04-25)
 2 [1] write about random        [waiting ] (2016-08-22)
 3 [1] write about sqlite3       [active  ] (2017-07-31)


### Query Metadata
> After cursor.execute() has been called,the cursor should set it's description attribute to hold information about data that will be returned by the fetch methods.
> 
> The description value is a sequence of tuples containing the colume name,tye,display size,internal size,precision,scale,and a flag says whether null value are accepted.

In [10]:
# sqlite3_cursor_description.py
import sqlite3

db_filename = 'todo.db'

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()

    cursor.execute("""
    select * from task where project = 'pymotw'
    """)

    print('Task table has these columns:')
    for colinfo in cursor.description:
        print(colinfo)

Task table has these columns:
('id', None, None, None, None, None, None)
('priority', None, None, None, None, None, None)
('details', None, None, None, None, None, None)
('status', None, None, None, None, None, None)
('deadline', None, None, None, None, None, None)
('completed_on', None, None, None, None, None, None)
('project', None, None, None, None, None, None)


### Row Object
> By default ,the values returned by the fetch methods as "rows" from the database are tuples.The Caller is responsible for knowing the order of the columns in the query and extracting indirvidual values from the tuple.
> 
> When the number of values in query grows,it is usually easier to work with an object and access value using their column names.Then the number and order of the tuple contents can change over time as the query is edited,and code depending on ther query results is less likely to break.
> 
> Connection objects have a row_factory property that allows the calling code to control the type of object created to represent each row in the query result set. sqlite3 also includes a Row class intended to be used as a row factory. Column values can be accessed through Row instances by using the column index or name.

In [11]:
# sqlite3_row_factory.py
import sqlite3

db_filename = 'todo.db'

with sqlite3.connect(db_filename) as conn:
    # Change the row factory to use Row
    conn.row_factory = sqlite3.Row

    cursor = conn.cursor()

    cursor.execute("""
    select name, description, deadline from project
    where name = 'pymotw'
    """)
    name, description, deadline = cursor.fetchone()

    print('Project details for {} ({})\n  due {}'.format(
        description, name, deadline))

    cursor.execute("""
    select id, priority, status, deadline, details from task
    where project = 'pymotw' order by deadline
    """)

    print('\nNext 5 tasks:')
    # Access the data with row[column name]
    for row in cursor.fetchmany(5):
        print('{:2d} [{:d}] {:<25} [{:<8}] ({})'.format(
            row['id'], row['priority'], row['details'],
            row['status'], row['deadline'],
        ))

Project details for Python Module of the Week (pymotw)
  due 2016-11-01

Next 5 tasks:
 1 [1] write about select        [done    ] (2016-04-25)
 2 [1] write about random        [waiting ] (2016-08-22)
 3 [1] write about sqlite3       [active  ] (2017-07-31)


### Query With Variables
> SQLite supports two forms for queries with placeholders, positional and named.

In [13]:
# sqlite3_argument_positional.py
# A question mark (?) denotes a positional argument,
# passed to execute() as a member of a tuple.

import sqlite3
import sys

db_filename = 'todo.db'
# project_name = sys.argv[1]
project_name = "pymotw"

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()

    query = """
    select id, priority, details, status, deadline from task
    where project = ?
    """

    cursor.execute(query, (project_name,))

    for row in cursor.fetchall():
        task_id, priority, details, status, deadline = row
        print('{:2d} [{:d}] {:<25} [{:<8}] ({})'.format(
            task_id, priority, details, status, deadline))
    

 1 [1] write about select        [done    ] (2016-04-25)
 2 [1] write about random        [waiting ] (2016-08-22)
 3 [1] write about sqlite3       [active  ] (2017-07-31)


In [14]:
# sqlite3_argument_named.py
import sqlite3
import sys

db_filename = 'todo.db'
# project_name = sys.argv[1]
project_name = "pymotw"

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()

    query = """
    select id, priority, details, status, deadline from task
    where project = :project_name
    order by deadline, priority
    """

    cursor.execute(query, {'project_name': project_name})

    for row in cursor.fetchall():
        task_id, priority, details, status, deadline = row
        print('{:2d} [{:d}] {:<25} [{:<8}] ({})'.format(
            task_id, priority, details, status, deadline))

 1 [1] write about select        [done    ] (2016-04-25)
 2 [1] write about random        [waiting ] (2016-08-22)
 3 [1] write about sqlite3       [active  ] (2017-07-31)


In [18]:
# sqlite3_argument_update.py
import sqlite3
import sys

db_filename = 'todo.db'
# id = int(sys.argv[1])
# status = sys.argv[2]
id = 2
status = "done"

with sqlite3.connect(db_filename) as conn:
    cursor = conn.cursor()
    query = "update task set status = :status where id = :id"
    cursor.execute(query, {'status': status, 'id': id})

# check updated task
!sqlite3 todo.db "select * from task"

1|1|write about select|done|2016-04-25||pymotw
2|1|write about random|done|2016-08-22||pymotw
3|1|write about sqlite3|active|2017-07-31||pymotw


### Bulk Loading
> To apply the same SQL instruction to a large set of data ,use executemany().

In [19]:
# sqlite3_load_csv.py
import csv
import sqlite3
import sys

db_filename = 'todo.db'
# data_filename = sys.argv[1]
data_filename = "data_filename.csv"

SQL = """
insert into task (details, priority, status, deadline, project)
values (:details, :priority, 'active', :deadline, :project)
"""

with open(data_filename, 'rt') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    with sqlite3.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.executemany(SQL, csv_reader)
        
!sqlite3 todo.db "select * from task"

1|1|write about select|done|2016-04-25||pymotw
2|1|write about random|done|2016-08-22||pymotw
3|1|write about sqlite3|active|2017-07-31||pymotw
4|2|finish reviewing markup|active|2016-11-30||pymotw
5|2|revise chapter intros|active|2016-08-20||pymotw
6|1|subtitle|active|2016-11-01||pymotw


### Define New Column Types
> SQLite has native support for integer, floating point, and text columns. Data of these types is converted automatically by sqlite3 from Python’s representation to a value that can be stored in the database, and back again, as needed. Integer values are loaded from the database into int or long variables, depending on the size of the value. Text is saved and retrieved as str, unless the text_factory for the Connection has been changed.
>
> Although SQLite only supports a few data types internally, sqlite3 includes facilities for defining custom types to allow a Python application to store any type of data in a column. Conversion for types beyond those supported by default is enabled in the database connection using the detect_types flag. Use PARSE_DECLTYPES if the column was declared using the desired type when the table was defined.